### 2.0. Imports

In [19]:
# Importing several libraries
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (6.0, 4.0) #setting figure size
import seaborn as sns
import matplotlib.dates as mdates
import os
import emoji
import re
pd.set_option('display.float_format', '{:.2f}'.format) # Set the float display option

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /Users/lgfolder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
DATAFILE = '../data/reddit_cleaned.csv'
df_all = pd.read_csv(DATAFILE)
df = df_all.head(50000).copy()
# df = df_all.copy()

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   50000 non-null  object 
 1   author               50000 non-null  object 
 2   created              50000 non-null  object 
 3   removed              50000 non-null  int64  
 4   deleted              50000 non-null  int64  
 5   is_self              50000 non-null  int64  
 6   is_video             50000 non-null  int64  
 7   title                50000 non-null  object 
 8   link_flair_text      50000 non-null  object 
 9   upvote_ratio         50000 non-null  float64
 10  score                50000 non-null  int64  
 11  num_comments         50000 non-null  int64  
 12  selftext             29228 non-null  object 
 13  shortlink            50000 non-null  object 
 14  FolderName           50000 non-null  object 
 15  word_count_selftext  50000 non-null 

In [22]:
#df.drop(['Unnamed: 0','Unnamed: 0.1'], axis = 1)

In [50]:
df['selftext'].fillna('notextprovided', inplace=True)

In [51]:
# Check data quality
print(f"Missing value: {df.isna().sum()}")
print(f"Duplicated rows: {df.duplicated().sum().sum()}")

Missing value: id                     0
author                 0
created                0
removed                0
deleted                0
is_self                0
is_video               0
title                  0
link_flair_text        0
upvote_ratio           0
score                  0
num_comments           0
selftext               0
shortlink              0
FolderName             0
word_count_selftext    0
word_count_title       0
date                   0
dtype: int64
Duplicated rows: 0


In [26]:
# Check class counts
df['selftext'].value_counts() 
# df.value_counts()

selftext
notextprovided                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### 2.x. Removing moderators ??? 

### 2.x. Spam messages - shall I remove them? 

### 2.x. TextBlob sentiment score

### 2.X. VADER sentiment score calculation 

In [52]:
# Instantiate an object of SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def calculate_sentiment(text):
    if pd.isnull(text):
        return {'neg': None, 'neu': None, 'pos': None, 'compound': None}
    
    # Analyze the sentiment of the post
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score

In [35]:
# This code checks the content of each inidividual text cell 
i = 1012
print(df["author"].loc[i])
print(df["selftext"].loc[i])
post = df["selftext"].loc[i]

# Calculates the sentiment scores for the chosen cell
overall_sentiment_score = analyzer.polarity_scores(post)
print(overall_sentiment_score)

HeyItsPixeL
Hi everyone. I am gonna be honest with you, I am really starting to get pissed about people spreading lies about what I said. I am getting tagged in about 150 - 200 posts a day and about the same amount of comments. Half of which is people trying to slander my name and telling other people lies about what I said is going to happen this week and how I supposedly said, this is "the last chance" (quote by one of the dumbf\*\*\* that slandered me over the last few weeks) for a Gamma Squeeze or Short Squeeze to happen.

&#x200B;

I am calling out this post specifically, because it gained the most traction and called out u/rensole and me:

[https:\/\/www.reddit.com\/r\/GME\/comments\/m5spf5\/reminder\_that\_the\_19th\_is\_not\_the\_last\_possible\/](https://preview.redd.it/qa1ogy259cn61.png?width=740&format=png&auto=webp&s=b479bb5b30085e2cf5c2ee345105dbb3ccdb9d18)

(Longer TL;DR at point III)

TL;DR: And I just want to clarify, because it actually seems to be a new FUD campaign (

In [36]:
df.index

RangeIndex(start=0, stop=50000, step=1)

In [53]:
# Apply the calculate_sentiment function to each row in the 'title' column (runtime = 2 seconds)
sentiments = df['title'].apply(calculate_sentiment)

# Record the sentiments into the dataframe (ttl=title, v= vader)
df.loc[:, 'neg_ttl_v'] = sentiments.apply(lambda x: x['neg'])
df.loc[:, 'neu_ttl_v'] = sentiments.apply(lambda x: x['neu'])
df.loc[:, 'pos_ttl_v'] = sentiments.apply(lambda x: x['pos'])
df.loc[:, 'compound_ttl_v'] = sentiments.apply(lambda x: x['compound'])

In [54]:
# Apply the calculate_sentiment function to each row in the 'selftext' column (runtime = 3 min+)
sentiments = df['selftext'].apply(calculate_sentiment)

# Record the sentiments into the dataframe (ttl=title, v= vader)
df.loc[:, 'neg_st_v'] = sentiments.apply(lambda x: x['neg'])
df.loc[:, 'neu_st_v'] = sentiments.apply(lambda x: x['neu'])
df.loc[:, 'pos_st_v'] = sentiments.apply(lambda x: x['pos'])
df.loc[:, 'compound_st_v'] = sentiments.apply(lambda x: x['compound'])

In [55]:
df.sample(5)

,id,author,created,removed,deleted,is_self,is_video,title,link_flair_text,upvote_ratio,...,word_count_title,date,neg_ttl_v,neu_ttl_v,pos_ttl_v,compound_ttl_v,neg_st_v,neu_st_v,pos_st_v,compound_st_v
12125,rssid0,NoMoneyNoMe,2021-12-31 12:04:49,0,0,1,0,How to profit from the upcoming war between NA...,Discussion,0.52,...,11,2021-12-31,0.25,0.57,0.18,-0.25,0.18,0.77,0.05,-0.93
40609,lcbahe,Suicidal_Donut,2021-02-04 08:30:59,0,0,1,0,What do you think people will do?,Discussion,0.82,...,7,2021-02-04,0.00,1.00,0.00,0.00,0.06,0.91,0.03,-0.51
10783,rpulu4,_STIFFL3R_,2021-12-27 19:03:39,0,0,0,0,Marry xmas and a happy new year!,Loss,0.96,...,7,2021-12-27,0.00,0.60,0.40,0.61,0.00,1.00,0.00,0.00
19135,l7xjio,leepawg,2021-01-29 16:54:43,1,0,1,0,At This Point It’s 🧻🤲🏽 vs 💎👐💎,Discussion,0.98,...,7,2021-01-29,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00
44750,n0o8wq,5EQiByvHDu,2021-04-28 20:39:18,0,0,1,0,Facebook's Q1 2021 Earnings: 48% Revenue Growt...,none,0.95,...,9,2021-04-28,0.00,0.76,0.24,0.38,0.01,0.83,0.16,0.97


### 2.x. Check the distribution of average VADER sentiment scores over time

In [ ]:
# Create a 4x2 grid of subplots
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(10, 10))
sentiment_titles = ['Negative Sentiment', 'Neutral Sentiment', 'Positive Sentiment', 'Compound Sentiment']

# The column pairs for sentiment values
sentiment_pairs = [('neg_st_v', 'neg_ttl_v'), 
                   ('neu_st_v', 'neu_ttl_v'), 
                   ('pos_st_v', 'pos_ttl_v'), 
                   ('compound_st_v', 'compound_ttl_v')]

for i, (sentiment_selftext, sentiment_title) in enumerate(sentiment_pairs):
    # Plot for 'selftext'
    axes[i, 0].hist(df[sentiment_selftext].dropna(), bins=50, color='skyblue', alpha=0.7)
    axes[i, 0].set_title(f'Selftext {sentiment_titles[i]}')
    axes[i, 0].set_xlabel('Score')
    axes[i, 0].set_ylabel('Frequency')
    
    # Plot for 'title'
    axes[i, 1].hist(df[sentiment_title].dropna(), bins=50, color='salmon', alpha=0.7)
    axes[i, 1].set_title(f'Title {sentiment_titles[i]}')
    axes[i, 1].set_xlabel('Score')
    # The ylabel only needs to be set on the leftmost plots
    if i == 0:
        axes[i, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Group by the 'Date' column and calculate the average of the 'compound' column
df_compound_by_date = df.groupby('date')['compound'].mean()
df_by_date = df.groupby('date')['compound'].mean()

# Display the result
df_compound_by_date
df_compound_by_date.plot.line()

### 2.x. Plotting trends in usage of emojis over time

In [ ]:
# Filter the DataFrame for rows where SELFTEXT contains the specified phrase/word/emoji
filtered_df = df[df['selftext'].str.contains("💎", na=False, regex=True)]
filtered_df

# Group by the 'Date' column and sum specific numerical columns
df_words_by_date = filtered_df.groupby('date')['id'].count()

# Display the result
df_words_by_date.plot.line()

In [ ]:
# Filter the DataFrame for rows where TITLE contains the specified phrase
filtered_df = df[df['title'].str.contains("💎", na=False, regex=True)]
filtered_df

# Group by the 'Date' column and sum specific numerical columns
df_words_by_date = filtered_df.groupby('date')['id'].count()
df_by_date = df.groupby('date')['id'].count()
df_words_by_date_pct = df_words_by_date / df_by_date * 100

# Display the result in PERCENTAGES
df_words_by_date_pct.plot.line()

In [ ]:
# List of words with bullish sentiment
bull_list = ['apes', 'DFV', 'hodling', 'HODL', 'diamond', 'gme', 'GME', 'moon', 'rocket']

# Escape each word in the bull list to ensure special regex characters are treated literally
escaped_bull_list = [re.escape(word) for word in bull_list]

# Create a regular expression pattern to match any of the words in the escaped bull list
pattern = r'\b(?:' + '|'.join(escaped_bull_list) + r')\b'

# Go through 'selftext' and 'title' for the presence of any of the words in the pattern
df.loc[:, 'bullflag'] = df['selftext'].str.contains(pattern, na=False, case=False) | \
                        df['title'].str.contains(pattern, na=False, case=False)


In [ ]:
# List of emojis with bullish sentiment
emoji_list = ['💎', '🦍', '🚀', '🙌']

# A function to create a valid column name for an emoji
def emoji_to_column_name(emoji):
    # Use a description or a numeric code that represents the emoji
    emoji_names = {
        '💎': 'e_diamond',
        '🦍': 'e_gorilla',
        '🚀': 'e_rocket',
        '🙌': 'e_hands'
    }
    return 'flag_' + emoji_names.get(emoji, 'emoji')

# Iterate over the list of emojis
for emoji in emoji_list:
    # Create a regular expression pattern for the current emoji
    pattern = re.escape(emoji)
    
    # Create a new column for each emoji in emoji_list
    column_name = emoji_to_column_name(emoji)
    df.loc[:, column_name] = df['selftext'].str.contains(pattern, na=False) | \
                             df['title'].str.contains(pattern, na=False)
    
    # Convert the boolean flags to integers to create dummy variables
    df.loc[:, column_name + '_dummy'] = df[column_name].astype(int)


In [ ]:
# List of columns to drop, all that don't end with '_dummy'
columns_to_drop = [col for col in df.columns if col.startswith('flag_') and not col.endswith('_dummy')]

# Drop the selected columns from the DataFrame
df = df.drop(columns=columns_to_drop)

In [ ]:
df.sample(5)

In [ ]:
# Group by 'date' and count the flagged rows.
date_grouped = df[df['bullflag']].groupby('date').size()

# Now we will get the total counts per 'date' in the original dataframe for normalization.
total_counts_by_date = df.groupby('date').size()

# Calculate the percentages.
percentages = (date_grouped / total_counts_by_date) * 100

# Plot the result in percentages.
percentages.plot.line()

In [ ]:

# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5), sharey=True)

# Define a function to filter, group, and plot the data based on the deletion status.
def plot_flagged_percentage_subplot(df, delete_status, ax):
    # Filter the DataFrame for the given delete_status and where 'flag' is True.
    flagged = df[(df['removed'] == delete_status) & df['bullflag']]
    
    # Group by 'date' and count the occurrences.
    flagged_by_date = flagged.groupby('date').size()
    
    # Calculate the percentages.
    percentages = (flagged_by_date / total_counts_by_date) * 100
    
    # Plot the percentages on the provided subplot axis.
    percentages.plot.line(ax=ax)
    ax.set_title(f"'deleted' = {delete_status}")
    ax.set_ylabel('Percentage')
    ax.set_xlabel('Date')

# Call the function for both deletion statuses with the respective subplot axis.
plot_flagged_percentage_subplot(df, 1, axes[0])  # For 'deleted' = 1
plot_flagged_percentage_subplot(df, 0, axes[1])  # For 'deleted' = 0

# Improve spacing between plots
plt.tight_layout()

# Display the plots
plt.show()


In [ ]:
percentages

In [ ]:
df.to_csv('../data/reddit_cleaned_vader_results.csv')

### 2.x. 

In [ ]:
df

### 2. Using regex for extracting ticker from the text

In [ ]:
# Modified code (need source forgot where it came from )

import re
import pandas as pd

# Function to find all occurrences of the stock ticker in the text blob
def find_occurrences_of_stock_ticker(arg_ticker, arg_text_to_search):
    # Case-insensitive search for the ticker and ticker prefixed with a '$'
    reg_ex_count = sum(1 for match in re.finditer(r'\b({}|${})\b'.format(arg_ticker, arg_ticker), 
                                                  arg_text_to_search, re.IGNORECASE))
    return reg_ex_count

# Load your tickers list from the CSV file
tickers_df = pd.read_csv('../data/tickers_list_short.csv')

# Assume df is your dataframe with a column 'selftext' containing the comments/posts
# Convert the 'selftext' to string to ensure that there are no floats
df['selftext'] = df['selftext'].astype(str)

# Create a text blob from all the comments/posts
text_blob = ' '.join(df['selftext'])

# Create a dictionary to hold the count of occurrences for each ticker
dictionary = {}

# Go through each ticker and count its occurrences in the text blob
for index, row in tickers_df.iterrows():
    ticker = row['Ticker']
    occurrences = find_occurrences_of_stock_ticker(ticker, text_blob)
    if occurrences > 0:
        dictionary[ticker] = occurrences

# Now you can sort the dictionary by occurrences and do further analysis
# For example, you can convert it to a DataFrame
ticker_counts_df = pd.DataFrame(list(dictionary.items()), columns=['Ticker', 'Occurrences']).sort_values(by='Occurrences', ascending=False)

# If you want to add this as a column to your original df, we can map the counts back to each comment
# We will create a function that maps the occurrences to each comment

def map_ticker_occurrences(comment, ticker_counts):
    tickers_in_comment = []
    for ticker, count in ticker_counts.items():
        if ticker in comment.upper():  # Converting to upper case for case-insensitive comparison
            tickers_in_comment.append(ticker)
    return tickers_in_comment

# Map the ticker occurrences back to the original DataFrame
df['tickers_mentioned'] = df['selftext'].apply(lambda comment: map_ticker_occurrences(comment, dictionary))

# Display the DataFrame with ticker counts
print(ticker_counts_df)

# Display the DataFrame with tickers mentioned in each comment
print(df[['selftext', 'tickers_mentioned']])

In [ ]:
# Map the ticker occurrences back to the original DataFrame
df['tickers_mentioned'] = df['selftext'].apply(lambda comment: map_ticker_occurrences(comment, dictionary))

# Display the DataFrame with ticker counts
print(ticker_counts_df)

# Display the DataFrame with tickers mentioned in each comment
print(df[['selftext', 'tickers_mentioned']])

In [ ]:
df

### 3.1. Ticker list selection
this code was adapted from here: https://github.com/theriley106/TheWSBIndex/tree/master

In [ ]:
# The original code written from pseudocode

import pandas as pd
import re

# Read the stock tickers and company names from a CSV file into a DataFrame
tickers_df = pd.read_csv('../data/tickers_list_short.csv')

# Create a dictionary to map both tickers and company names to their official ticker symbols
# Mapping is done by converting everything to uppercase to ensure case-insensitive matching
# It also handles NaN values by checking if the entries are not null
ticker_name_map = {
    str(row['Ticker']).upper(): row['Ticker'] 
    for _, row in tickers_df.iterrows() if pd.notnull(row['Ticker'])
}
ticker_name_map.update({
    str(row['Name']).upper(): row['Ticker'] 
    for _, row in tickers_df.iterrows() if pd.notnull(row['Name'])
})

# Define the process_comment function that will take a comment string and return a dictionary
# containing categorized tickers based on the action words found in the comment
def process_comment(comment):
    # If comment is not a string (e.g., NaN), return an empty dictionary
    if not isinstance(comment, str):
        return {'puts': [], 'calls': [], 'buy': [], 'sell': []}

    # Initialize a dictionary to store categorized tickers
    comment_info = {'puts': [], 'calls': [], 'buy': [], 'sell': []}

    # Convert the entire comment to uppercase to match against the ticker_name_map
    comment_upper = comment.upper()
    
    # Use regex to find all words in the comment
    words = re.findall(r'\b\w+\b', comment_upper)

    # Initialize an empty list to temporarily store tickers as we find them
    tempList = []

    # Iterate through each word in the comment
    for word in words:
        # Check if the word is a ticker or company name, adjusting for $ prefix and matching against the map
        if word in ticker_name_map or word.strip('$') in ticker_name_map:
            # If it is, add the official ticker symbol to tempList
            ticker = ticker_name_map.get(word, ticker_name_map.get(word.strip('$'), ''))
            tempList.append(ticker)
        # Check for buying action words and update the comment_info dictionary
        elif word in ['BUY', 'BUYING']:
            comment_info['buy'].extend(tempList)
            tempList.clear()
        # Check for selling action words and update the comment_info dictionary
        elif word in ['SELL', 'SOLD', 'CLOSE', 'CLOSING', 'SHORTS']:
            comment_info['sell'].extend(tempList)
            tempList.clear()
        # Check for 'puts' and 'calls' action words and update the comment_info dictionary
        elif word == 'PUTS' and tempList:
            comment_info['puts'].extend(tempList)
            tempList.clear()
        elif word == 'CALLS' and tempList:
            comment_info['calls'].extend(tempList)
            tempList.clear()

    # Handle any remaining tickers that haven't been categorized
    if tempList:
        # By default, we add any uncategorized tickers to the 'buy' category
        # Modify as needed based on your logic or requirements
        comment_info['buy'].extend(tempList)

    # Return the populated dictionary for this comment
    return comment_info

# Fill NaN values with an empty string before applying the process_comment function
df['selftext'] = df['selftext'].fillna('')
df['comment_info'] = df['selftext'].apply(process_comment)

# Apply the process_comment function only to non-null rows in the 'selftext' column
# If 'selftext' is NaN, the corresponding 'comment_info' will be an empty dictionary
df['comment_info'] = df['selftext'].dropna().apply(process_comment)

# Optionally, if you want to create separate columns for each action
# Expand the dictionaries in 'comment_info' into separate columns
df[['puts', 'calls', 'buy', 'sell']] = df['comment_info'].apply(pd.Series)

# Display the DataFrame to verify the output
print(df[['selftext', 'puts', 'calls', 'buy', 'sell']])

In [ ]:
df['sell'].value_counts(normalize=True).head(20)

In [ ]:
# code to extract stock ticker mentions from the social media posts 

import pandas as pd
import re

# Sample text containing stock mentions
text = "I just bought $GME and $AAPL, and I'm considering $AMZN as well. $gme is on fire! GME, Gamestop, gamestop, GameStop, gme."

# Sample list of publicly traded tickers
ticker_data = {
    'Ticker': ['$GME', '$AAPL', '$AMZN', '$TSLA'],
    'Company': ['GameStop Inc.', 'Apple Inc.', 'Amazon.com Inc.', 'Tesla, Inc.']
}

# Create a DataFrame from the ticker data
ticker_df = pd.DataFrame(ticker_data)

# Define a regular expression pattern to find stock mentions
pattern = r'\$?(?i)\b(' + '|'.join(map(re.escape, ticker_df['Ticker'])) + r')\b'

# Find all stock mentions in the text
mentions = re.findall(pattern, text)

# Remove duplicate mentions
unique_mentions = list(set(mentions))

# Create a dictionary to store the mentions and associated companies
mentions_dict = {}
for mention in unique_mentions:
    company = ticker_df[ticker_df['Ticker'].str.lower() == mention.lstrip('$').lower()]['Company'].values[0]
    mentions_dict[mention] = company

# Create a DataFrame from the mentions dictionary
mentions_df = pd.DataFrame.from_dict(mentions_dict, orient='index', columns=['Company'])

# Print the extracted mentions
print(mentions_df)


In [ ]:
from collections import Counter

# Define a function to get the most common ticker
def get_most_common_ticker(tickers_list):
    if not tickers_list:  # If the list is empty
        return None  # Or you could return a default string like 'No Tickers'
    # Count the tickers and get the most common one
    most_common_ticker = Counter(tickers_list).most_common(1)[0][0]
    return most_common_ticker

# Apply this function to each row in the 'buy' column to create a new 'signal' column
df['buy_signal'] = df['buy'].apply(get_most_common_ticker)
df['buy_signal'].value_counts(normalize=True)

# Apply this function to each row in the 'buy' column to create a new 'signal' column
df['sell_signal'] = df['sell'].apply(get_most_common_ticker)
df['sell_signal'].value_counts(normalize=True)

In [ ]:
# This script connects to reddit's API and retrieves a count of stock $ticker mentions to gauge rising popularity.
# from https://github.com/DrakeDavis/RedditApiStockParser/blob/main/parse_tickers_from_reddit.py
import praw
import json
import time
import datetime
from datetime import timezone
import re
import boto3
import os
import pytz
import sys

# Function to find all occurrences of the stock ticker (or $ticker) with case ignored. Returns the number of occurrences
def find_occurrences_of_stock_ticker(arg_ticker, arg_text_to_search):
    # Regex that also checks for boundaries (start of sentence, end of sentence, etc.)
    reg_ex_count = sum(1 for match in re.finditer(r"\b{}\b".format(arg_ticker), arg_text_to_search, re.IGNORECASE))

    # Also check for ticker with a $ in front of it
    prefaced_ticker = "$" + arg_ticker
    reg_ex_count = reg_ex_count + sum(1 for match in re.finditer(r"\b{}\b".format(prefaced_ticker), arg_text_to_search,
                                                                 re.IGNORECASE))
    return reg_ex_count


# Connection credentials to reddit's API
reddit = praw.Reddit(
    client_id=os.environ['REDDIT_API_CLIENT_ID'],
    client_secret=os.environ['REDDIT_API_CLIENT_SECRET'],
    user_agent=os.environ['REDDIT_API_USER_AGENT']
)

# Instantiating objects
posts_in_last_day = []
text_blob = ''

# Retrieve subreddit name from terminal argument
subreddit_name = str(sys.argv[1])

# Get all posts from subreddit in the last 24 hours (limit is 900, but no 24 period has reached that number)
for post in reddit.subreddit(subreddit_name).new(limit=900):
    post_title = post.title
    post_creation_epoch_time = post.created - 60 * 60 * 8  # subtracting 8 hours due to timezone
    current_epoch_time = int(time.time())
    age_of_post_in_hours = (current_epoch_time - post_creation_epoch_time) / 60 / 60

    if age_of_post_in_hours < 24:
        posts_in_last_day.append(post)

# Define metrics for posts and comments in the last 24 hours
post_count_in_last_day = posts_in_last_day.__len__()
comments_in_last_day = 0

# Retrieve all comments from the acquired posts
for post in posts_in_last_day:
    text_blob = text_blob + post.title
    post.comments.replace_more(limit=1)
    for comment in post.comments.list():
        if comment.body:
            comments_in_last_day = comments_in_last_day + 1
            text_blob = text_blob + comment.body

# The text_blob is an amalgamation of all posts and comments from the last 24 hours
# We're going to parse it and find occurrences of stock names
dictionary = {}
with open("curated_stock_tickers.txt") as f:
    for line in f:
        line = line.rstrip('\n')
        print("Currently counting: " + str(line))
        occurrences = find_occurrences_of_stock_ticker(line, text_blob)
        if occurrences > 0:
            dictionary[line] = occurrences

# Get the current time and format it accordingly
current_time = datetime.datetime.now(timezone.utc)
est = pytz.timezone('US/Eastern')
date_format = "%d %B %I:%M %p"

# Write out the data in .json format for consumption by the frontend
json_data = {"posts": post_count_in_last_day, "comments": comments_in_last_day,
             "time": current_time.astimezone(est).strftime(date_format),
             "data": (sorted(dictionary.items(), key=lambda x: x[1], reverse=True))}
fp = open(subreddit_name + '_most_mentioned_stocks.json', 'w+')
fp.write(json.dumps(json_data))
fp.close()

# Open connection to AWS S3 bucket
s3 = boto3.resource('s3',
                    aws_access_key_id=os.environ['S3_KEY'],
                    aws_secret_access_key=os.environ['S3_SECRET'])
s3_client = boto3.client('s3',
                         aws_access_key_id=os.environ['S3_KEY'],
                         aws_secret_access_key=os.environ['S3_SECRET'])

# Upload the .json file to S3. Making it public so anyone can use it.
s3_client.upload_file(subreddit_name + '_most_mentioned_stocks.json', 'wsb-pop-index',
                        subreddit_name + 'PopIndex.json', ExtraArgs={'ContentType': "application/json",
                        'ACL': 'public-read'})

In [ ]:
df['selftext', ].values[105]

### 2.x. Lemmatization

Options: Spark NLP? 

### 2.x. Vectorisation

CountVectorizer and HashingTF from PySpark to transform textual data into vectors. 

### 3.x. Classifier

### 3.x. Clustering

k-means++ clustering from both PySpark's machine learning library and scikit-learn, combined with PCA and TruncatedSVD/t-SNE from scikit-learn for dimensionality reduction